In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

import requests 
import json
from shutil import which
import time
import os
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import urllib.request
import itertools

import dateutil.parser as dparser
from datetime import datetime

In [2]:
class nuevoliving():
    def __init__(self, items = list(), headless = False):
        
        self.items = items
        self.headless = headless
        chrome_options = Options()

        if self.headless:
            chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-infobars")
        chrome_options.add_argument("start-maximized")
        chrome_options.add_argument("--disable-extensions")

        prefs = {"profile.default_content_setting_values.notifications" : 2}
        chrome_options.add_experimental_option("prefs",prefs)

        chrome_path = r"C:\Users\chaitanya\OneDrive\Documents\Moro\Data scrapping\chromedriver_win32\chromedriver.exe"
        global driver
        driver = webdriver.Chrome(executable_path=chrome_path ,options=chrome_options)
        driver.get("https://www.nuevoliving.com/us/customer/account/login/")
        time.sleep(1)
        search_email = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, "//input[@title = 'Email Address']")))
        search_email.click()

        search_email.send_keys("jose@moro.com")
        # print('email entered')


        search_pass = driver.find_element(By.XPATH, "//input[@title = 'Password']")
        search_pass.click()
        # time.sleep(3)

        search_pass.send_keys("nuevoguest")

        search_pass.send_keys(Keys.ENTER)

    
    def scrap(self, category):
        time.sleep(5)
        cat = f"https://www.nuevoliving.com/us/{category}"
        driver.get(cat)

#         items = []

        exit = True
        while exit:
            products = driver.find_elements(By.XPATH, "//div[@class = 'category-products']/ul/li/h2/div/a")

            for product in products:
                product_details = {}
                product_details['product_url'] = product.get_attribute('href')
                href = product_details['product_url'] 

                driver.execute_script("window.open('" +href +"');")
                # switch to new window
                driver.switch_to.window(driver.window_handles[1])
                name = []
                name_paths = driver.find_elements(By.XPATH, "//div[@class = 'product-name']/h1/span")
                for opt in name_paths:
                    name.append(opt.text)
                
                product_details['name'] = ' '.join(name)
                try:
                    sku_path = driver.find_element(By.XPATH, "//div[@class = 'product-sku']")
                    product_details['sku'] = sku_path.text
                except:
                    continue
                    
                try:
                    avaliable_path = driver.find_element(By.XPATH, "//span[@class = 'qtyonhand']/p/span[2]")
                    product_details['avaliable_new'] = avaliable_path.text
                except:
                    product_details['avaliable_new'] = None
                print(product_details)
                self.items.append(product_details)


                time.sleep(3)
                driver.close()
                #back to main window
                driver.switch_to.window(driver.window_handles[0])

            try:
                next_page_path = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='toolbar-bottom']/div/div[@class = 'pager']/div/ol/li/a[@class = 'next i-next']")))
                next_page = next_page_path.get_attribute('href')

                driver.get(next_page)
                time.sleep(1)
            except:
                exit = False
        return self.items
        

In [63]:
# data_json = []
start = nuevoliving(data_json, False)
categories = ['products/table', 'products/seating', 'products/sofa', 'products/modular-sofa',\
              'products/bedroom', 'products/media-storage', 'products/lighting', 'district-eight/seating', \
              'district-eight/sofa', 'district-eight/tables', 'district-eight/bedroom', 'district-eight/games', 'catalogsearch/result/?q=Drift+shelf']

# categories = ['catalogsearch/result/?q=Drift+shelf']
for category in categories:
    data = start.scrap(category)

C:\Users\CHAITA~1\AppData\Local\Temp/ipykernel_6132/427539646.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path ,options=chrome_options)


{'product_url': 'https://www.nuevoliving.com/us/drift-6838', 'name': 'DRIFT MEDIA UNIT', 'sku': 'HGDA827', 'avaliable_new': '9'}
{'product_url': 'https://www.nuevoliving.com/us/drift-6847', 'name': 'DRIFT SIDE TABLE', 'sku': 'HGDA836', 'avaliable_new': '5'}
{'product_url': 'https://www.nuevoliving.com/us/drift-6839', 'name': 'DRIFT MEDIA UNIT', 'sku': 'HGDA828', 'avaliable_new': '10'}
{'product_url': 'https://www.nuevoliving.com/us/drift-6389', 'name': 'DRIFT SHELF', 'sku': 'HGDA546', 'avaliable_new': '21'}
{'product_url': 'https://www.nuevoliving.com/us/drift-6842', 'name': 'DRIFT SHELF', 'sku': 'HGDA831', 'avaliable_new': '18'}
{'product_url': 'https://www.nuevoliving.com/us/drift-6392', 'name': 'DRIFT SHELF', 'sku': 'HGDA659', 'avaliable_new': 'OUT OF STOCK'}
{'product_url': 'https://www.nuevoliving.com/us/drift-6844', 'name': 'DRIFT SHELF', 'sku': 'HGDA833', 'avaliable_new': '13'}
{'product_url': 'https://www.nuevoliving.com/us/drift-6828', 'name': 'DRIFT SIDE TABLE', 'sku': 'HGDA8

In [64]:
len(data)

2038

In [69]:
import pandas as pd
import numpy as np

df = pd.DataFrame(data, )
df


,product_url,name,sku,avaliable_new
0,https://www.nuevoliving.com/us/aida,AIDA SIDE TABLE,HGDJ955,16
1,https://www.nuevoliving.com/us/aiden,AIDEN DINING TABLE,HGNA436,7
2,https://www.nuevoliving.com/us/aiden-8518,AIDEN DINING TABLE,HGNA577,11-30-22
3,https://www.nuevoliving.com/us/aiden-8504,AIDEN DINING TABLE,HGNA451,10-10-22
4,https://www.nuevoliving.com/us/aiden-8509,AIDEN DINING TABLE,HGNA456,11-30-22
...,...,...,...,...
2033,https://www.nuevoliving.com/us/drift-6841,DRIFT SIDEBOARD,HGDA830,11
2034,https://www.nuevoliving.com/us/drift-6390,DRIFT SHELF,HGDA545,20
2035,https://www.nuevoliving.com/us/drift-6843,DRIFT SHELF,HGDA832,18
2036,https://www.nuevoliving.com/us/drift-6827,DRIFT DESK,HGDA801,8


In [70]:
df['avaliable_new'].unique()

array(['16', '7', '11-30-22', '10-10-22', '11-01-22', '8', '62',
       '11-05-22', '21', '10-30-22', 'OUT OF STOCK', '2', '6', '22', '11',
       '10-15-22', '1', '08-30-22', '01-05-23', '28', '66', '9', '65',
       '30', '10', '09-10-22', '29', '5', '143', '46', '10-20-22', '64',
       '08-20-22', '14', '25', '23', '11-20-22', '40', '26', '17', '12',
       '15', '19', '3', '18', '12-10-22', '09-15-22', '39', '13', '42',
       '27', '44', '32', '4', '20', '01-06-23', '10-18-22', '33', '35',
       '67', '34', '154', '09-20-22', '09-09-22', '31', '89', '12-15-22',
       '11-15-22', '58', '63', '91', '37', '105', '78', '113', '41',
       '218', '111', '229', '192', '241', '10-11-22', '10-05-22', '55',
       '54', '80', '124', '82', '10-01-22', '74', '121', '110', '24',
       '47', '112', '93', '75', '36', '52', '128', '163', '176', '133',
       '09-01-22', '43', '387', '51', '09-30-22', '72', '76', '59', '87',
       '10-08-22', '84', '60', '108', '109', '107', '73', '95', '50'

In [71]:
df.drop_duplicates(subset = ['sku'], keep='last', inplace = True, ignore_index=True)

In [72]:
df

,product_url,name,sku,avaliable_new
0,https://www.nuevoliving.com/us/aida,AIDA SIDE TABLE,HGDJ955,16
1,https://www.nuevoliving.com/us/aiden,AIDEN DINING TABLE,HGNA436,7
2,https://www.nuevoliving.com/us/aiden-8518,AIDEN DINING TABLE,HGNA577,11-30-22
3,https://www.nuevoliving.com/us/aiden-8504,AIDEN DINING TABLE,HGNA451,10-10-22
4,https://www.nuevoliving.com/us/aiden-8509,AIDEN DINING TABLE,HGNA456,11-30-22
...,...,...,...,...
1766,https://www.nuevoliving.com/us/drift-6841,DRIFT SIDEBOARD,HGDA830,11
1767,https://www.nuevoliving.com/us/drift-6390,DRIFT SHELF,HGDA545,20
1768,https://www.nuevoliving.com/us/drift-6843,DRIFT SHELF,HGDA832,18
1769,https://www.nuevoliving.com/us/drift-6827,DRIFT DESK,HGDA801,8


In [73]:
df['estimated_date'] = None
df['quantity'] = None
for i in range(len(df)):
    try:
        if (len(df['avaliable_new'][i])) >3 and (df['avaliable_new'][i] != 'OUT OF STOCK'):
            df['estimated_date'][i] = df['avaliable_new'][i]
    except:
        pass
    try:
        df['quantity'][i] = int(df['avaliable_new'][i])
    except:
        pass

In [74]:
df.loc[7]

product_url       https://www.nuevoliving.com/us/aiden-8502
name                                     AIDEN DINING TABLE
sku                                                 HGNA439
avaliable_new                                      10-10-22
estimated_date                                     10-10-22
quantity                                               None
Name: 7, dtype: object

In [75]:
df['avaliable_new'].unique()

array(['16', '7', '11-30-22', '10-10-22', '8', '11-01-22', '62',
       '11-05-22', '21', '10-30-22', 'OUT OF STOCK', '2', '6', '22', '11',
       '10-15-22', '1', '08-30-22', '01-05-23', '28', '66', '9', '65',
       '30', '10', '09-10-22', '29', '5', '143', '10-20-22', '46', '64',
       '08-20-22', '14', '25', '23', '11-20-22', '40', '26', '17', '12',
       '15', '19', '3', '18', '12-10-22', '09-15-22', '39', '13', '42',
       '27', '44', '32', '4', '20', '01-06-23', '10-18-22', '33', '35',
       '67', '34', '154', '09-20-22', '09-09-22', '31', '89', '12-15-22',
       '11-15-22', '58', '63', '91', '37', '105', '78', '113', '41',
       '218', '111', '229', '192', '241', '10-11-22', '10-05-22', '54',
       '124', '82', '55', '80', '10-01-22', '74', '121', '110', '24',
       '47', '112', '93', '75', '36', '52', '128', '163', '176', '133',
       '09-01-22', '43', '387', '51', '09-30-22', '72', '59', '76', '87',
       '10-08-22', '84', '60', '108', '109', '107', '73', '95', '50'

In [76]:

for i in range(len(df)):
    try:
        if df['quantity'][i] is not None:
            df['avaliable_new'][i] =  '5-7 business days'
        elif "OUT OF STOCK" in df['avaliable_new'][i]:
            df['avaliable_new'][i] = '10-11 weeks'
        else:
            no_of_days = int((dparser.parse(df['avaliable_new'][i] ,fuzzy=True)  - datetime.today()).days)

            if no_of_days<=7:
                df['avaliable_new'][i] = '5-7 business days'
            elif no_of_days > 7 and no_of_days <60:
                df['avaliable_new'][i] = f'{no_of_days//7}-{no_of_days//7 +1} weeks'
            else:
                df['avaliable_new'][i] = f'{no_of_days//7 -1}-{no_of_days//7} weeks'
    except:
        print(i)

In [77]:
df.iloc[1749]

product_url       https://www.nuevoliving.com/us/theodore-6878
name                                 THEODORE OCCASIONAL CHAIR
sku                                                    HGTB582
avaliable_new                                5-7 business days
estimated_date                                            None
quantity                                                    28
Name: 1749, dtype: object

In [78]:
if "OUT OF STOCK" in df['avaliable_new'][107]:
    print('yo')

In [79]:
df.to_excel("nuevoliving2.xlsx", index=False)

In [26]:
df

,product_url,name,sku,avaliable_new,estimated_date,quantity
0,https://www.nuevoliving.com/us/aida,AIDA SIDE TABLE,HGDJ955,5-7 business days,None,16
1,https://www.nuevoliving.com/us/aiden,AIDEN DINING TABLE,HGNA436,5-7 business days,None,7
2,https://www.nuevoliving.com/us/aiden-8518,AIDEN DINING TABLE,HGNA577,13-14 weeks,11-30-22,None
3,https://www.nuevoliving.com/us/aiden-8504,AIDEN DINING TABLE,HGNA451,6-7 weeks,10-10-22,None
4,https://www.nuevoliving.com/us/aiden-8509,AIDEN DINING TABLE,HGNA456,13-14 weeks,11-30-22,None
...,...,...,...,...,...,...
1744,https://www.nuevoliving.com/us/foosball,FOOSBALL TABLE,HGDA242,5-7 business days,None,4
1745,https://www.nuevoliving.com/us/shuffleboard-6778,SHUFFLEBOARD TABLE,HGDA617,10-11 weeks,None,None
1746,https://www.nuevoliving.com/us/ping-pong-table...,PING PONG TABLE,HGDA556,5-7 business days,None,2
1747,https://www.nuevoliving.com/us/ping-pong-table,PING PONG TABLE,HGDA494,5-7 business days,02-09-22,None


In [41]:
df1 = pd.read_excel('nuevoliving1.xlsx')


In [44]:
len(df1)

1581

In [42]:
df_full = pd.concat([df1, df], ignore_index=True)
df_full

,product_url,name,sku,avaliable_new,quantity
0,https://www.nuevoliving.com/us/aida,AIDA SIDE TABLE,HGDJ955,5 -7 business days,16.0
1,https://www.nuevoliving.com/us/aiden-8517,AIDEN DINING TABLE,HGNA576,9 - 10 weeks,NaN
2,https://www.nuevoliving.com/us/aiden-8503,AIDEN DINING TABLE,HGNA440,14 - 15 weeks,NaN
3,https://www.nuevoliving.com/us/aiden-8522,AIDEN DINING TABLE,HGNA581,9 - 10 weeks,NaN
4,https://www.nuevoliving.com/us/aiden-8508,AIDEN DINING TABLE,HGNA455,7 - 8 weeks,NaN
...,...,...,...,...,...
1720,https://www.nuevoliving.com/us/foosball,FOOSBALL TABLE,HGDA242,5 -7 business days,4
1721,https://www.nuevoliving.com/us/shuffleboard-6778,SHUFFLEBOARD TABLE,HGDA617,10 - 11 weeks,None
1722,https://www.nuevoliving.com/us/ping-pong-table...,PING PONG TABLE,HGDA556,5-7 business days,None
1723,https://www.nuevoliving.com/us/ping-pong-table,PING PONG TABLE,HGDA494,5-7 business days,None


In [43]:
df_full.to_excel("nuevoliving_date.xlsx", index=False)